In [1]:
'''
Code written entirely by Professor Christopher Horvat of Brown University.
Classifier network to detect whether wave-ice interactions will result in fracture (to remove degenerate data points).
Uses a dense feed-forward network with two layers, using a binary cross-entropy loss.
'''

import numpy as np
import matplotlib.pyplot as plt
import math
import random
from scipy.io import loadmat
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras.utils import to_categorical
from numpy import argmax
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model


In [2]:
include_last = True;

#locstr = '/Users/chorvat/Dropbox (Brown)/Research Projects/Active/FSTD-Ongoing-Work/Neural-Net-Waves/'
locstr = '/content/drive/Shared drives/DeepWIFF/'
training_string = locstr + 'Training_Data/training_data_errtol'

if not include_last:
    training_string = training_string + '_nolast'

training_string = training_string + '.mat'

architecture = [(100,100)]


In [3]:
training_data = loadmat(training_string)
misc_data = loadmat(locstr + 'Training_Data/NN_params.mat')

FileNotFoundError: ignored

In [ ]:
input_vec = training_data['input_vec'].T
output_vec = training_data['output_vec'].T

Freq = misc_data['f']
dFreq = Freq * (np.sqrt(1.1) - np.sqrt(1/1.1)); 
Redge = misc_data['redge'].T
Rcent = misc_data['rcent'].T
Lambda = misc_data['lambda']

In [ ]:
ice_thick = input_vec[:,-1]
wave_spec = input_vec[:,:25]
# wave_spec[wave_spec < 10**-8] = 0
wave_energy = np.sum(wave_spec*np.expand_dims(dFreq,axis=1).T,axis=1)
peak_loc = np.argmax(wave_spec,axis=1)
peak_freq = Freq[peak_loc]


num_frac = np.sum(output_vec,axis=1)
usable = num_frac > 0
print('Out of ' + str(len(num_frac)) + ', ' + str(sum(usable)) + ' can be used')


classifier_input = input_vec # ,peak_freq]).T
classifier_output = to_categorical(usable)

# print(classifier_output[:10])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(classifier_input,classifier_output,test_size=0.3,stratify=classifier_output)


In [ ]:
for structno in range(len(architecture)):

    struct = architecture[structno]
    
    print('Size is ' + str(struct))

    classifier = Sequential();
    #First Hidden Layer
    classifier.add(Dense(struct[0], activation='relu', kernel_initializer='random_normal', input_dim=26));
    print('Adding layer with size ' + str(struct[0]))

    #Other Hidden Layers
    for i in range(len(struct)-1):
        print('Adding layer with size ' + str(struct[i+1]))
        classifier.add(Dense(struct[i+1], activation='relu', kernel_initializer='random_normal'));

    classifier.add(Dense(2, activation='softmax', kernel_initializer='random_normal'));

    #Output Layer
    classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy']);

    # Output name
    outstr = 'Class-'

    for i in range(len(struct)):
        outstr = outstr + str(struct[i]) + 'x'

    outstr = outstr[:-1]

    if not include_last:
        outstr = outstr + '-nolast'        

    fname = 'Classifiers/' + outstr + '-FINAL.h5'

    print(fname)
    
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
    
    classifier.fit(X_train,y_train,validation_data = (X_test,y_test),batch_size=256, epochs=250) #,callbacks = [es])
    
    classifier.save(fname)

In [ ]:
thresh = 0.5 

y_pred_class =classifier.predict(X_test)

In [ ]:
y_true = y_test[:,1]
y_pred = y_pred_class[:,1] > thresh

cm = confusion_matrix(y_true, y_pred)
print(cm)

tn, fp, fn, tp = cm.ravel()

fnr = fn / [fn + tp]
fpr = fp / [fp + tn]

In [ ]:
print('false positive rate is ' + str(round(100*fpr[0],2)) + '%')
print('false negative rate is ' + str(round(100*fnr[0],2)) + '%')

In [ ]:
nthresh = 200

thresher = np.linspace(0,1,nthresh)
fnr = np.zeros(nthresh)
fpr = np.zeros(nthresh)
toter = np.zeros(nthresh)
reduc = np.zeros(nthresh)
y_true = y_test[:,1]

for i in range(nthresh):
    
    y_pred = np.multiply((y_pred_class[:,1] > thresher[i]),1)
    
    tp = sum(y_pred*y_true)
    tn = sum((1-y_pred)*(1-y_true))
    fn = sum((1-y_pred)*y_true)
    fp = sum(y_pred*(1-y_true))

    toter[i] = (fn+fp)/(len(y_true))
    fnr[i] = fn / [fn + tp]
    fpr[i] = fp / [fp + tn]
    reduc[i] = (tn + fn)/len(y_true)
    
sumerate = fpr + fnr
minsumthresh = np.argmin(sumerate)
minthresh = np.argmin(toter)

    

In [ ]:
plt.plot(thresher,fnr)
plt.plot(thresher,fpr)
plt.plot(thresher,fpr+fnr)
plt.plot(thresher,toter)
plt.legend(('FNR','FPR','Sum','Error Rate'))
plt.grid()
plt.ylim((0,.2))
plt.scatter(thresher[minsumthresh],sumerate[minsumthresh])
plt.scatter(thresher[minthresh],sumerate[minthresh])

optreduc = (sum(1-y_true))/len(y_true)

print('Best Sum Error is ' + str(round(sumerate[minsumthresh]*100,2)) + '% At threshold of ' + str(round(thresher[minsumthresh],2)))
print('Corresponds to FNR of ' + str(round(100*fnr[minsumthresh],2)) + '%')
print('Corresponds to FPR of ' + str(round(100*fpr[minsumthresh],2)) + '%')
print('Reduces by ' + str(round(100*reduc[minsumthresh],2)) + '%')
print('Optimal reduction is' + str(round(100*optreduc)) + '%')
print('Efficiency is ' + str(round(100*reduc[minsumthresh]/optreduc)))
